In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
dataset

# Cálculo do prior de morte

In [ ]:
priorDeath = dataset['DEATH_EVENT'].value_counts(normalize=True)

priorYesDeath = priorDeath[0]
priorNoDeath  = priorDeath[1]

In [ ]:
##converter df em serie com value_count e denovo em df, com as colunas corretas
def mountProbabilityDataframeForColumn(desiredColumn, df):
    valueSeries = df[[desiredColumn,"DEATH_EVENT"]].value_counts(normalize=True)
    df_value_counts = pd.DataFrame(valueSeries)
    ## não sei pq isso deveria ser necessário?
    df_value_counts = df_value_counts.reset_index()
    df_value_counts.columns = [desiredColumn, "DEATH_EVENT", "probability"] # change column names
    return df_value_counts

In [ ]:
##criando os p(x | Death)

chances = {}


chances['age'] = mountProbabilityDataframeForColumn("age", dataset)
chances['anaemia'] = mountProbabilityDataframeForColumn("anaemia", dataset)
chances['creatinine_phosphokinase'] = mountProbabilityDataframeForColumn("creatinine_phosphokinase", dataset)
chances['diabetes'] = mountProbabilityDataframeForColumn("diabetes", dataset)
chances['ejection_fraction'] = mountProbabilityDataframeForColumn("ejection_fraction", dataset)
chances['high_blood_pressure'] = mountProbabilityDataframeForColumn("high_blood_pressure", dataset)
chances['platelets'] = mountProbabilityDataframeForColumn("platelets", dataset)
chances['serum_creatinine'] = mountProbabilityDataframeForColumn("serum_creatinine", dataset)
chances['serum_sodium'] = mountProbabilityDataframeForColumn("serum_sodium", dataset)
chances['sex'] = mountProbabilityDataframeForColumn("sex", dataset)
chances['smoking'] = mountProbabilityDataframeForColumn("smoking", dataset)
chances['time'] = mountProbabilityDataframeForColumn("time", dataset)
        
chances



In [ ]:
##poc da estrutura de dados
todasAsIdades = chances['age']

sim = todasAsIdades[todasAsIdades["DEATH_EVENT"] == 0]

simIdade = sim[sim["age"] == 60]

probIdade = simIdade["probability"]

print(probIdade[0])

In [ ]:
## retorna pSim, pNao para um dado valor em coluna
def getProbListForGivenParameterAndColumn(column, value, chances):
    allValues = chances[column]
    sim = allValues.loc[(allValues["DEATH_EVENT"] == 1) & (allValues[column] == value), 'probability'].to_list()
    nao =  allValues.loc[(allValues["DEATH_EVENT"] == 0) & (allValues[column] == value), 'probability'].to_list()
    print("Coluna atual: "+column)
    retornoSim = 0.001
    retornoNao = 0.001

    if(sim):
        retornoSim = sim[0]
    if(nao):
        retornoNao = nao[0]
    

    return retornoSim, retornoNao

probAge = getProbListForGivenParameterAndColumn("age", 60, chances )

probAge


In [ ]:
dataset

## Exemplo: 50 anos, com anemia, 146 creatinina, com diabetes, ejection_fraction 45, sem pressao alta, 210000.00 plaquetas, creatinina 1,9, serum_sodio 130, sex 1, smoking 1, time 3

In [ ]:
# Calcular o (sim, nao) para cada tipo de variavel

age = getProbListForGivenParameterAndColumn("age", 50, chances)
anaemia = getProbListForGivenParameterAndColumn("anaemia", 1, chances)
creatinine_phosphokinase = getProbListForGivenParameterAndColumn("creatinine_phosphokinase", 146, chances)
diabetes = getProbListForGivenParameterAndColumn("diabetes", 1, chances)
ejection_fraction = getProbListForGivenParameterAndColumn("ejection_fraction", 45, chances)
high_blood_pressure = getProbListForGivenParameterAndColumn("high_blood_pressure", 0, chances)
platelets = getProbListForGivenParameterAndColumn("platelets", 210000.00, chances)
serum_creatinine = getProbListForGivenParameterAndColumn("serum_creatinine", 1.9, chances)
serum_sodium = getProbListForGivenParameterAndColumn("serum_sodium", 130, chances)
sex = getProbListForGivenParameterAndColumn("sex", 1, chances)
smoking = getProbListForGivenParameterAndColumn("smoking", 1, chances)
time = getProbListForGivenParameterAndColumn("time", 3, chances)


In [ ]:
##chance dele não morrer, em log2
pNaoMorrer = priorNoDeath * age[0] * anaemia[0] * creatinine_phosphokinase[0] * diabetes[0] * ejection_fraction[0] * high_blood_pressure[0] * platelets[0] * serum_creatinine[0] * serum_sodium[0] * sex[0] * smoking[0] * time[0]
print(math.log2(pNaoMorrer))

In [ ]:
##chance dele morrer, em log2
pMorrer = priorYesDeath * age[1] * anaemia[1] * creatinine_phosphokinase[1] * diabetes[1] * ejection_fraction[1] * high_blood_pressure[1] * platelets[1] * serum_creatinine[1] * serum_sodium[1] * sex[1] * smoking[1] * time[1]
print(math.log2(pMorrer))

In [ ]:
print ('Morrer : ', pMorrer)
print ('Não morrer :  ', pNaoMorrer)


print("sobreviver: ",pNaoMorrer / (pNaoMorrer+ pMorrer))

print("morrer: ",pMorrer / (pNaoMorrer+ pMorrer))